In [1]:
import urllib.request

import geopandas
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import pandas as pd
import planetary_computer
import pystac_client
import rioxarray
import xarray as xr
import odc.stac
import dask.distributed

In [2]:
from shapely import wkt

In [3]:
# https://boundingbox.klokantech.com/

region = wkt.loads(
    "POLYGON((-58.611677 -34.469254, -58.271327 -34.469254, -58.271327 -34.725584, -58.611677 -34.725584, -58.611677 -34.469254))"
)
bbox = region.bounds

In [4]:
start = pd.Timestamp("2020-01-01")
stop = pd.Timestamp("2020-01-03")

In [5]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
    modifier=planetary_computer.sign_inplace,
)

filter_query = {"s5p:processing_mode": {"eq": "OFFL"}, "s5p:product_name": {"eq": "no2"}}

query = catalog.search(
    collections=["sentinel-5p-l2-netcdf"],
    datetime=[start, stop],
    limit=100,  # Consider making this configurable
    bbox=bbox,
    query=filter_query,
)

items = list(query.items())

In [12]:
items[0]

<Item id=S5P_L2_NO2____20200102T172846_20200102T191016_11511>

In [6]:
len(items)

3

In [7]:
import fsspec

In [8]:
!pip install h5netcdf


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [14]:
f = fsspec.open(items[0].assets["no2"].href).open()

In [15]:
ds = xr.open_dataset(f, group="PRODUCT", engine="h5netcdf")
ds

<xarray.Dataset> Size: 331MB
Dimensions:                                               (scanline: 4172,
                                                           ground_pixel: 450,
                                                           time: 1, corner: 4,
                                                           polynomial_exponents: 6,
                                                           intensity_offset_polynomial_exponents: 1,
                                                           layer: 34,
                                                           vertices: 2)
Coordinates:
  * scanline                                              (scanline) float64 33kB ...
  * ground_pixel                                          (ground_pixel) float64 4kB ...
  * time                                                  (time) datetime64[ns] 8B ...
  * corner                                                (corner) float64 32B ...
  * polynomial_exponents                                  (polynomial_exponents) float64 48B ...
  * intensity_offset_polynomial_exponents                 (intensity_offset_polynomial_exponents) float64 8B ...
  * layer                                                 (layer) float64 272B ...
  * vertices                                              (vertices) float64 16B ...
    latitude                                              (time, scanline, ground_pixel) float32 8MB ...
    longitude                                             (time, scanline, ground_pixel) float32 8MB ...
Data variables:
    delta_time                                            (time, scanline) datetime64[ns] 33kB ...
    time_utc                                              (time, scanline) object 33kB ...
    qa_value                                              (time, scanline, ground_pixel) float32 8MB ...
    nitrogendioxide_tropospheric_column                   (time, scanline, ground_pixel) float32 8MB ...
    nitrogendioxide_tropospheric_column_precision         (time, scanline, ground_pixel) float32 8MB ...
    nitrogendioxide_tropospheric_column_precision_kernel  (time, scanline, ground_pixel) float32 8MB ...
    averaging_kernel                                      (time, scanline, ground_pixel, layer) float32 255MB ...
    air_mass_factor_troposphere                           (time, scanline, ground_pixel) float32 8MB ...
    air_mass_factor_total                                 (time, scanline, ground_pixel) float32 8MB ...
    tm5_tropopause_layer_index                            (time, scanline, ground_pixel) float64 15MB ...
    tm5_constant_a                                        (layer, vertices) float32 272B ...
    tm5_constant_b                                        (layer, vertices) float32 272B ...

In [37]:
list(ds.variables)

['scanline',
 'ground_pixel',
 'time',
 'corner',
 'polynomial_exponents',
 'intensity_offset_polynomial_exponents',
 'layer',
 'vertices',
 'latitude',
 'longitude',
 'delta_time',
 'time_utc',
 'qa_value',
 'nitrogendioxide_tropospheric_column',
 'nitrogendioxide_tropospheric_column_precision',
 'nitrogendioxide_tropospheric_column_precision_kernel',
 'averaging_kernel',
 'air_mass_factor_troposphere',
 'air_mass_factor_total',
 'tm5_tropopause_layer_index',
 'tm5_constant_a',
 'tm5_constant_b']

In [19]:
from geocube.vector import vectorize

In [20]:
variable='nitrogendioxide_tropospheric_column'
gdf = vectorize(ds[variable])

In [21]:
gdf.head()

,nitrogendioxide_tropospheric_column,geometry
0,-0.000008,"POLYGON ((0.5 221.5, 0.5 222.5, 1.5 222.5, 1.5..."
1,-0.000007,"POLYGON ((0.5 222.5, 0.5 223.5, 1.5 223.5, 1.5..."
2,-0.000018,"POLYGON ((1.5 222.5, 1.5 223.5, 2.5 223.5, 2.5..."
3,-0.000012,"POLYGON ((2.5 222.5, 2.5 223.5, 3.5 223.5, 3.5..."
4,-0.000002,"POLYGON ((0.5 223.5, 0.5 224.5, 1.5 224.5, 1.5..."


In [22]:
intersected = gdf[gdf.intersects(region)]

In [26]:
intersected.geometry.values[:5]

<GeometryArray>
[]
Length: 0, dtype: geometry